<a href="https://colab.research.google.com/github/deepakanna/Deep_Learning/blob/main/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [59]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer

In [60]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [61]:
!wget --no-check-certificate \
    https://drive.google.com/uc?id=13ySLC_ue6Umt9RJYSeM2t-V0kCv-4C-P -O /tmp/sentiment.csv

--2023-04-04 05:37:33--  https://drive.google.com/uc?id=13ySLC_ue6Umt9RJYSeM2t-V0kCv-4C-P
Resolving drive.google.com (drive.google.com)... 172.217.203.100, 172.217.203.138, 172.217.203.101, ...
Connecting to drive.google.com (drive.google.com)|172.217.203.100|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-08-ak-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/aijp4p5k83206sv5ef16fqf1b2n81ssp/1680586650000/11118900490791463723/*/13ySLC_ue6Umt9RJYSeM2t-V0kCv-4C-P?uuid=4b52d714-4162-4685-8298-e03340242d55 [following]
--2023-04-04 05:37:33--  https://doc-08-ak-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/aijp4p5k83206sv5ef16fqf1b2n81ssp/1680586650000/11118900490791463723/*/13ySLC_ue6Umt9RJYSeM2t-V0kCv-4C-P?uuid=4b52d714-4162-4685-8298-e03340242d55
Resolving doc-08-ak-docs.googleusercontent.com (doc-08-ak-docs.googleusercontent.com)... 142.251.162.132, 2607:f8b0:400c:c38::84
Connecting to d

In [62]:
import pandas as pd
import numpy as np

In [63]:
df=pd.read_csv('/tmp/sentiment.csv')
df.head()

,Unnamed: 0,text,sentiment
0,0,So there is no way for me to plug it in here i...,0
1,1,Good case Excellent value.,1
2,2,Great for the jawbone.,1
3,3,Tied to charger for conversations lasting more...,0
4,4,The mic is great.,1


In [64]:
label=df['sentiment'].tolist()

In [65]:
sent=df['text'].tolist()
vocab=1000

In [66]:
tokenizer=Tokenizer(num_words=vocab,oov_token='oov')
tokenizer.fit_on_texts(sent)

In [67]:
training_size=int(len(sent)*0.8)
training_set=sent[:training_size]
testing_set=sent[training_size:]
training_label=label[0:training_size]
testing_label=label[training_size:]


In [68]:
seq=tokenizer.texts_to_sequences(training_set)
max_length=50
padded_seq=pad_sequences(seq,maxlen=max_length,padding='post',truncating='post')

In [74]:
seq1=tokenizer.texts_to_sequences(testing_set)
max_length=50
padded_seq1=pad_sequences(seq1,maxlen=max_length,padding='post',truncating='post')

In [75]:
tr_label

array([0, 1, 1, ..., 1, 0, 1])

In [76]:
tr_label=np.array(training_label)
tst_label=np.array(testing_label)

In [77]:
dim=16
model=tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab,dim,input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(6,activation='relu'),
    tf.keras.layers.Dense(1,activation='sigmoid')

])
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 50, 16)            16000     
                                                                 
 global_average_pooling1d_3   (None, 16)               0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_6 (Dense)             (None, 6)                 102       
                                                                 
 dense_7 (Dense)             (None, 1)                 7         
                                                                 
Total params: 16,109
Trainable params: 16,109
Non-trainable params: 0
_________________________________________________________________


In [81]:
epochs=30
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
model.fit(padded_seq,tr_label,epochs=epochs,validation_data=(padded_seq1,tst_label))

Epoch 1/30
50/50 [==============================] - 2s 13ms/step - loss: 0.6933 - accuracy: 0.4959 - val_loss: 0.6939 - val_accuracy: 0.4110
Epoch 2/30
50/50 [==============================] - 0s 7ms/step - loss: 0.6927 - accuracy: 0.5223 - val_loss: 0.6946 - val_accuracy: 0.4110
Epoch 3/30
50/50 [==============================] - 0s 4ms/step - loss: 0.6917 - accuracy: 0.5574 - val_loss: 0.6902 - val_accuracy: 0.6341
Epoch 4/30
50/50 [==============================] - 0s 3ms/step - loss: 0.6884 - accuracy: 0.6748 - val_loss: 0.6884 - val_accuracy: 0.6667
Epoch 5/30
50/50 [==============================] - 0s 3ms/step - loss: 0.6831 - accuracy: 0.7131 - val_loss: 0.6823 - val_accuracy: 0.7368
Epoch 6/30
50/50 [==============================] - 0s 4ms/step - loss: 0.6741 - accuracy: 0.7878 - val_loss: 0.6706 - val_accuracy: 0.7444
Epoch 7/30
50/50 [==============================] - 0s 4ms/step - loss: 0.6618 - accuracy: 0.7690 - val_loss: 0.6591 - val_accuracy: 0.7519
Epoch 8/30
50/50 [=

In [94]:
model1=tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab,dim,input_length=max_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(16,return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(16)),
    tf.keras.layers.Dense(6,activation='relu'),
    tf.keras.layers.Dense(1,activation='sigmoid')
])
model1.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
model1.fit(padded_seq,tr_label,epochs=epochs,validation_data=(padded_seq1,tst_label))

Epoch 1/30
50/50 [==============================] - 16s 134ms/step - loss: 0.6918 - accuracy: 0.5210 - val_loss: 0.6947 - val_accuracy: 0.4135
Epoch 2/30
50/50 [==============================] - 4s 77ms/step - loss: 0.6623 - accuracy: 0.6227 - val_loss: 0.5854 - val_accuracy: 0.7193
Epoch 3/30
50/50 [==============================] - 5s 105ms/step - loss: 0.3855 - accuracy: 0.8412 - val_loss: 0.4955 - val_accuracy: 0.7845
Epoch 4/30
50/50 [==============================] - 4s 75ms/step - loss: 0.2598 - accuracy: 0.9052 - val_loss: 0.4700 - val_accuracy: 0.7870
Epoch 5/30
50/50 [==============================] - 4s 78ms/step - loss: 0.2051 - accuracy: 0.9247 - val_loss: 0.4633 - val_accuracy: 0.8045
Epoch 6/30
50/50 [==============================] - 5s 102ms/step - loss: 0.1523 - accuracy: 0.9485 - val_loss: 0.4425 - val_accuracy: 0.8095
Epoch 7/30
50/50 [==============================] - 4s 77ms/step - loss: 0.1263 - accuracy: 0.9598 - val_loss: 0.5316 - val_accuracy: 0.8020
Epoch 8/3

In [103]:
fake_reviews = ["I love this phone", 
                "Everything was cold",
                "Everything was hot exactly as I wanted", 
                "Everything was green", 
                "the host seated us immediately",
                "they gave us free chocolate cake", 
                "I hate u",
                "The phone is expensive but easy to use than any other",
                "we couldn't hear each other talk because of the shouting in the kitchen"
              ]

In [104]:
f_sent=tokenizer.texts_to_sequences(fake_reviews)
padded_seq2=pad_sequences(f_sent,maxlen=max_length,padding='post',truncating='post')

In [105]:
classes=model1.predict(padded_seq2)

1/1 [==============================] - 0s 33ms/step


In [106]:
for i in range(len(fake_reviews)):
  print(fake_reviews[i])
  print(classes[i])

I love this phone
[0.99726146]
Everything was cold
[0.00089762]
Everything was hot exactly as I wanted
[0.6749954]
Everything was green
[0.00202014]
the host seated us immediately
[0.9973365]
they gave us free chocolate cake
[0.99735427]
I hate u
[0.00079676]
The phone is expensive but easy to use than any other
[0.9978517]
we couldn't hear each other talk because of the shouting in the kitchen
[0.00053113]
